In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
SPREADSHEET_ID = os.getenv("EXPENSES_SPREADHSEET_ID")

In [ ]:
months = ["enero", "febrero", "marzo", "abril", "mayo", "junio", "julio"]
month_urls = {month: f"https://docs.google.com/spreadsheets/d/{SPREADSHEET_ID}/gviz/tq?tqx=out:csv&sheet={month}" for month in months}

grupos = {
    "fijos": ["alquileres", "servicios_esenciales", "servicios_no_esenciales"],
    "corrientes": ["hogar", "transporte", "salidas"],
    "irregulares": ["shopping", "otros"],
}

In [ ]:
totales_mensuales = []

for mes in months:
    url = month_urls[mes]
    df = pd.read_csv(
        url, usecols=grupos["fijos"] + grupos["corrientes"] + grupos["irregulares"]
    )
    fila_total = df.iloc[-1]
    fila_limpia = {
        cat: pd.to_numeric(
            fila_total[cat].replace("$", "").replace(",", "").strip(), errors="coerce"
        )
        for cat in grupos["fijos"] + grupos["corrientes"] + grupos["irregulares"]
    }
    fila_limpia["mes"] = mes
    totales_mensuales.append(fila_limpia)
df_totales = pd.DataFrame(totales_mensuales).set_index("mes")
df_totales.tail(12)

In [ ]:
def plot_grupo(grupo, nombre):
    df_grupo = df_totales[grupo]
    df_grupo.plot(marker="o", figsize=(10, 5))
    plt.title(f"Gastos mensuales: {nombre}")
    plt.ylabel("Monto en ARS")
    plt.xlabel("Mes")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()


plot_grupo(grupos["fijos"], "Gastos Fijos")
plot_grupo(grupos["corrientes"], "Gastos Corrientes")
plot_grupo(grupos["irregulares"], "Gastos Irregulares")

trimestres = {month: f"Q{(i // 3) + 1}" for i, month in enumerate(months)}

df_totales["trimestre"] = df_totales.index.map(trimestres)

promedios_trimestrales = {}
for nombre_grupo, categorias in grupos.items():
    df_prom = df_totales.groupby("trimestre")[categorias].mean()
    promedios_trimestrales[nombre_grupo] = df_prom

for nombre_grupo, df_prom in promedios_trimestrales.items():
    df_prom.plot(kind="bar", stacked=True, figsize=(8, 5))
    plt.title(f"Promedio Trimestral - Grupo {nombre_grupo.capitalize()}")
    plt.ylabel("Monto promedio por categoría (ARS)")
    plt.xlabel("Trimestre")
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:
datos_personales = pd.read_csv("../data/sueldos.csv")
datos_personales_del_anio = datos_personales.iloc[-len(months):]
total_anual = df_totales.sum(numeric_only=True).sum()
ingreso_anual = datos_personales_del_anio["sueldo_neto_ars"].sum()
ahorro_uso_discrecional = ingreso_anual - total_anual
porcentaje_gastado = total_anual / ingreso_anual * 100

print(f"Gasto total del año: ${total_anual:,.0f} ARS")
print(f"Ingreso neto del año: ${ingreso_anual:,.0f} ARS")
print(f"Diferencia: ${ahorro_uso_discrecional:,.0f} ARS")
print(f"Porcentaje del ingreso gastado: {porcentaje_gastado:.1f}%")